In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import os
from pathlib import Path
import re

In [2]:
def get_gaps(list_freqs,gap_width_tolerance:float=0.005):
    flattened_lambdas = np.sort(np.concatenate(list_freqs))
    bands = []
    gaps_list = np.diff(flattened_lambdas)
    for i,gaps in enumerate(gaps_list):
        bandgap_lower = flattened_lambdas[i]
        bandgap_upper = flattened_lambdas[i + 1]
        gap_width = (bandgap_upper-bandgap_lower)/((bandgap_upper+bandgap_lower)/2)
        if gap_width>=gap_width_tolerance:
            bands.append([bandgap_lower,bandgap_upper,gap_width])
    return bands

In [3]:
directory = "../Luis Data/SHU 2D point patterns (N=200)/CTL_Files"

In [ ]:
extracted_data = {}
for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
        if filename.endswith(".csv"):
            print(f"Processing file: {filename}")
            file_path = os.path.join(dirpath, filename)
            type_sim = "_".join(Path(dirpath).parts[-2:])
            file_key = Path(filename).stem + "_" + type_sim
            if file_key not in extracted_data.keys():
                extracted_data[file_key]={}
            freqs=[]
            k_points=[]
            with open(file_path, newline='') as csvfile: 
                reader = csv.reader(csvfile)
                for row in reader:
                    freqs.append(row[5:])
                    k_points.append(row[0])
                extracted_data[file_key]["freqs"] = freqs
                extracted_data[file_key]["k_points"] = k_points

Processing file: chi_0.30_posics_Lx_14.1421347_Ly_14.1421347_N_200.0_rods_sample_1_out.csv
Processing file: chi_0.30_posics_Lx_14.1421347_Ly_14.1421347_N_200.0_rods_sample_2_out.csv
Processing file: chi_0.30_posics_Lx_14.1421347_Ly_14.1421347_N_200.0_rods_sample_3_out.csv
Processing file: chi_0.30_posics_Lx_14.1421347_Ly_14.1421347_N_200.0_rods_sample_4_out.csv
Processing file: chi_0.30_posics_Lx_14.1421347_Ly_14.1421347_N_200.0_rods_sample_5_out.csv
Processing file: chi_0.31_posics_Lx_13.9283876_Ly_13.9283876_N_194.0_rods_sample_1_out.csv
Processing file: chi_0.31_posics_Lx_13.9283876_Ly_13.9283876_N_194.0_rods_sample_2_out.csv
Processing file: chi_0.31_posics_Lx_13.9283876_Ly_13.9283876_N_194.0_rods_sample_3_out.csv
Processing file: chi_0.31_posics_Lx_13.9283876_Ly_13.9283876_N_194.0_rods_sample_4_out.csv
Processing file: chi_0.31_posics_Lx_13.9283876_Ly_13.9283876_N_194.0_rods_sample_5_out.csv
Processing file: chi_0.32_posics_Lx_14.0356684_Ly_14.0356684_N_197.0_rods_sample_1_out.csv

In [5]:
#directory to save figures
gap_list = {}
save_fig_path = "./figures_mpb_bands"
os.makedirs(save_fig_path,exist_ok=True)
for item in extracted_data.keys():
    if len(extracted_data[item]["freqs"])==0:
        continue
    freqs = np.array(extracted_data[item]["freqs"][1:],dtype=float)
    k_points= np.array(extracted_data[item]["k_points"][1:],dtype=float)

    for freq_item in freqs.T:
        plt.scatter(k_points-1,(freq_item), color="blue",facecolors='none')

    tick_labs = [r'$\Gamma$', r'$X$']
    plt.ylabel('frequency (c/a)', size=16)
    points_in_between = 5
    tick_locs = [i*points_in_between+i for i in range(len(tick_labs))]
    title = item
    chi = float(re.search(r'chi_([+-]?\d+(?:\.\d+)?)', title).group(1))
    sample = float(re.search(r'sample_([+-]?\d+(?:\.\d+)?)', title).group(1))
    gap = np.array(get_gaps(freqs[:,100:]))
    try:
        gap_item = gap[np.argmax(gap[:, 2])]
    except:
        gap_item=[0,0,0]

    plt.fill_between(k_points-1,gap_item[0],gap_item[1], color='red', alpha=0.2)
    plt.text(0.5,((gap_item[0]+gap_item[1])/2), rf'$\Delta \omega / \omega_0 = {gap_item[2]*100 :.2f}$%', fontsize = 12)

    plt.grid(True)
    plt.xticks(tick_locs, tick_labs)
    
    plt.title("sample: "+str(sample)+", chi: "+str(chi), size=16)
    if chi not in gap_list.keys():
        gap_list[chi] = {}
    if f"sample_{sample}" not in gap_list[chi].keys():
        gap_list[chi][f"sample_{sample}"] = {"gap_edges":[], "gap_center":[],"gap_width":[]}
    gap_list[chi][f"sample_{sample}"]["gap_width"].append(gap_item[2])
    gap_list[chi][f"sample_{sample}"]["gap_edges"].append([gap_item[0],gap_item[1]])
    gap_list[chi][f"sample_{sample}"]["gap_center"].append((gap_item[0]+gap_item[1])/2)
    # plt.show()
    # raise Exception("Stop")
    os.makedirs(f"{save_fig_path}/{chi}",exist_ok=True)
    plt.savefig(rf"{save_fig_path}/{chi}/{sample}.png")
    plt.close()

In [6]:
# Assuming /AutomationModule is in the root directory of your project
import sys
import os
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))
from AutomationModule import * 

import AutomationModule as AM

os.makedirs("./data",exist_ok=True)

In [7]:
AM.create_hdf5_from_dict(gap_list,"./data/20251215_eps_11p56_chi_0p3_0p37_bgw_list.h5")